<a href="https://colab.research.google.com/github/suri-pu-bi/LLM-Study/blob/main/%08chap1_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

* **5일 동안의 날씨 데이터를 보고, 다음 날의 기온을 예측하는 RNN**
* 각 시점에서 이전 시점의 은닉 상태를 사용하여 다음 시점의 출력을 계산
* 입력 데이터 : 각 샘플은 5일동안의 날씨 데이터, 10개의 특성(온도, 습도, 풍속 등)
* 출력 데이터 : 다음 날의 기온 - 전체 데이터를 바탕으로 예측한 값
* hidden_size : 은닉 상태의 차원 = 은닉 상태가 얼마나 많은 정보를 저장할 수 있는지



In [2]:
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # 초기 은닉 상태 설정
        # RNN의 층 수, 시간단계의 수, 각 시간단계에 대한 은닉 상태 크기
        h0 = torch.zeros(1, x.size(1), self.hidden_size)

        # RNN에 x와 초기 은닉 상태 전달
        out, _ = self.rnn(x, h0)

        # 마지막 단계의 출력을 분류기에 전달
        out = self.fc(out[:, -1, :])
        return out

In [3]:
# 입력 및 출력 사이즈 정의
input_size = 10 # 각 시간단계에서 입력되는 특성의 수
hidden_size = 20
output_size = 1

# 임의의 훈련 데이터 생성
# 100개의 샘플, 각 샘플은 5단계의 시계열 데이터를 가짐, 각 시간 단계마다 10개의 특성을 가짐
x_train = torch.randn(100, 5, input_size)
y_train = torch.randn(100, 1)

# 테스트 데이터
x_test = torch.randn(20, 5, input_size)
y_test = torch.randn(20, 1)

In [4]:
# 모델 초기화
model = SimpleRNN(input_size, hidden_size, output_size)

criterion = nn.MSELoss()
# 옵티마이저: 계산된 기울기를 기반으로 매개변수를 업데이트
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [5]:
x_train[0]

tensor([[-5.4039e-01, -8.9587e-01,  7.1640e-01,  1.6035e-01,  8.7199e-02,
          2.7400e-01, -1.9559e-01, -8.1648e-01, -5.0449e-01,  6.9159e-01],
        [-2.7241e-01,  5.3045e-01,  4.6342e-02, -2.9757e-01,  1.4681e+00,
         -4.5258e-01, -1.8807e+00, -2.2232e-01, -1.3155e+00, -7.5496e-01],
        [-2.4267e-01, -2.4859e-01,  5.9883e-01, -1.0414e+00,  5.5553e-01,
          5.0237e-01,  1.6144e+00,  1.7261e+00,  8.1371e-01, -9.7624e-02],
        [-2.6406e-01,  2.9176e-01, -5.7686e-01, -7.3417e-02,  4.5261e-01,
          3.6611e-02,  1.1064e+00, -1.9697e+00, -2.5203e-01, -6.7232e-01],
        [-6.4719e-01,  6.8748e-01, -1.8605e+00, -4.5622e-04, -1.8284e-01,
          7.0304e-01,  4.9998e-01, -8.6345e-01, -1.9611e+00,  5.4862e-01]])

In [6]:
x_train[0].unsqueeze(0).shape

torch.Size([1, 5, 10])

In [7]:
y_train[0]

tensor([-1.3882])

In [8]:
# 랜덤한 라벨을 사용했으므로 loss값 큼 ..!
epochs = 10

for epoch in range(epochs):
    for i in range(len(x_train)):
        # 현재 배치 데이터
        inputs = x_train[i].unsqueeze(0) # 배치 차원 추가
        labels = y_train[i].unsqueeze(0)

        # forward
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [1/10], Loss: 0.0112
Epoch [2/10], Loss: 0.0108
Epoch [3/10], Loss: 0.0114
Epoch [4/10], Loss: 0.0128
Epoch [5/10], Loss: 0.0068
Epoch [6/10], Loss: 0.0054
Epoch [7/10], Loss: 0.0089
Epoch [8/10], Loss: 0.0075
Epoch [9/10], Loss: 0.0076
Epoch [10/10], Loss: 0.0086


In [9]:
# 모델을 평가 모드로 설정
model.eval()

test_loss = 0
with torch.no_grad():
    for i in range(len(x_test)):
        inputs = x_test[i].unsqueeze(0)
        labels = y_test[i].unsqueeze(0)

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

    print(f'Test Loss: {test_loss / len(x_test):.4f}')

Test Loss: 1.2962
